In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import cvxpy as cp
import torch
from cvxpylayers.torch import CvxpyLayer

%load_ext autoreload
%autoreload 2

In [ ]:
def dx(v, w, t, eps=1e-15):
    return v * np.sin((w + eps) * t) / (w + eps)

def dy(v, w, t, eps=1e-15):
    return v * (1 - np.cos((w + eps) * t)) / (w + eps)

def f(v, w, t, eps=1e-15):
    return np.array([dx(v, w, t, eps), dy(v, w, t, eps)])

In [ ]:
# Convex approximation for sine from pi to 2pi
def sin_taylor(x):
    # Taylor expansion of sine around 3*pi/2
    # sin(x) = sin(3*pi/2) + cos(3*pi/2) * (x - 3*pi/2) - sin(3*pi/2) * (x - 3*pi/2)^2 / 2
    #        = -1 + 0 * (x - 3*pi/2) - (-1) * (x - 3*pi/2)^2 / 2
    #        = -1 + (x - 3*pi/2)^2 / 2
    return -1 + (x - 3*np.pi/2)**2 / 2


# Plot sin_taylor
x = np.linspace(np.pi, 2*np.pi, 100)
plt.plot(x, np.sin(x), label='sin')
plt.plot(x, sin_taylor(x), label='sin_taylor')
plt.legend()
plt.show()

In [ ]:
T = 3.0
c = np.array([1, 0])


v = cp.Variable(1)
w = cp.Variable(1)
t = cp.Variable(1)

x = cp.Variable(1)
constraints = [np.pi <= x, x <= 2*np.pi]
#objective = cp.Minimize(cp.pnorm(f(v, w, t) - c, p=2))
objective = cp.Minimize(sin_taylor(x))
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

In [ ]:
#cvxpylayer = CvxpyLayer(problem, parameters=[v, w], variables=[t])
v_tch = torch.randn(1, requires_grad=True)
w_tch = torch.randn(1, requires_grad=True)

# solve the problem
solution, = cvxpylayer(v_tch, w_tch)

# compute the gradient of the sum of the solution with respect to A, b
solution.sum().backward()